# Serverless Example
## PyTorch [CLASSIFICATION]

## Setup

In [1]:
# Install some dependencies
!pip install torch


import time
import logging
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from sklearn import datasets
from sklearn.model_selection import train_test_split

# Use CPU or GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.__version__

'1.1.0'

### Load Database

In [0]:
iris = datasets.load_iris()

X = iris.data.astype('float32')
y = iris.target.astype('int')

## Training Process

### DataLoaders definition
Data Loader is a client function for the bulk import of data. It reads the contentent and divides it in batches. We divide the dataset into training e validation sets.

In [3]:
BATCH_SIZE = 8

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)
print('Training examples    : {:5d}'.format(len(X_train)))
print('Validation examples  : {:5d}'.format(len(X_valid)))

train_ds = DataLoader(dataset=TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train)),
                      batch_size=BATCH_SIZE, 
                      shuffle=True)

valid_ds = DataLoader(dataset=TensorDataset(torch.from_numpy(X_valid), torch.from_numpy(y_valid)),
                      batch_size=BATCH_SIZE, 
                      shuffle=False)

Training examples    :   120
Validation examples  :    30


### Model Definition
We define a simple neural network. In PyTorch we need to specify the input size ("in_features" parameter of the first layer).

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(in_features=4, out_features=5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.log_softmax(self.fc3(x), dim=1)

In [0]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1)

### Training
The model will give bad performances due to the lack of normalization of the inputs. We keep saving the model that gives the best performances during training.

In [0]:
# Evaluation metric
# It is calculated as the mean 
# of the metric through the batch

def eval(data_loader, model):
    # Cumulative loss
    cum_loss = 0
    num_examples = 0
    
    for data, label in data_loader:
        data = data.to(device)
        label = label.to(device)
        num_examples += data.shape[0]
        
        y_pred = model(data)
        loss = F.nll_loss(y_pred, label)
        cum_loss += loss.item()
    return cum_loss/num_examples

In [7]:
EPOCHS = 10
MODEL_PATH = 'model'

train_loss = []    
valid_loss = []

# Time to train
for e in range(EPOCHS):
    tick = time.time()
    
    # Batch training
    model.train()
    for data, label in train_ds:
        data = data.to(device)
        label = label.to(device)
        
        # Backpropagation
        optimizer.zero_grad()
        y_pred = model(data)
        loss = F.nll_loss(y_pred, label)
        loss.backward()
        optimizer.step()
        
    # Training metrics
    train_loss.append(eval(train_ds, model))
    
    # Validation metrics
    valid_loss.append(eval(valid_ds, model))
    
    # Save the model if it reaches the best performances
    if valid_loss[-1] == min(valid_loss):
        torch.save(model, f'{MODEL_PATH}.pt')
    
    print('Epoch {:3d} [{:.2f} sec] - Training Loss: {:.4f} - Validation Loss: {:.4f}'.format(e+1, time.time()-tick, train_loss[e], valid_loss[e]))

Epoch   1 [0.05 sec] - Training Loss: 0.1314 - Validation Loss: 0.1401


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch   2 [0.03 sec] - Training Loss: 0.1137 - Validation Loss: 0.1232
Epoch   3 [0.02 sec] - Training Loss: 0.0867 - Validation Loss: 0.0953
Epoch   4 [0.02 sec] - Training Loss: 0.0781 - Validation Loss: 0.0857
Epoch   5 [0.03 sec] - Training Loss: 0.0734 - Validation Loss: 0.0807
Epoch   6 [0.02 sec] - Training Loss: 0.0702 - Validation Loss: 0.0770
Epoch   7 [0.03 sec] - Training Loss: 0.0686 - Validation Loss: 0.0754
Epoch   8 [0.02 sec] - Training Loss: 0.0666 - Validation Loss: 0.0730
Epoch   9 [0.03 sec] - Training Loss: 0.0657 - Validation Loss: 0.0722
Epoch  10 [0.03 sec] - Training Loss: 0.0654 - Validation Loss: 0.0695


## Testing Process
We simulate a test phase in which we only have a trained model saved in the current directory.

### Blank Paper

In [0]:
# Lets put us in blank paper condition
del model

### Prediction
Load the model and simulate to predict the whole iris dataset.

In [9]:
logger = logging.getLogger('iris')


## Prediction 
def handle(event, **kwargs):
    # If data is received as json convert to pandas
    event = event['data'] if 'data' in event else event
    if not isinstance(event, pd.DataFrame):
        event = pd.DataFrame.from_dict(event, orient='columns')

    # Convert to NDArray
    data = torch.from_numpy(event.values.astype('float32'))
    
    # Retrieve model from disk and use it for predictions
    model = torch.load(f'{MODEL_PATH}.pt')
    model.eval()
    
    # Target format convertion
    target_dict = {0: 'setosa', 1: 'versicolor', 2:'virginica'}
    to_target = np.vectorize(lambda x: target_dict[x])
    
    return to_target(np.argmax(model(data).detach().numpy(), axis=1)).tolist()

## Testing and liveness check
def test(data, **kwargs):
    pred = handle(data)

    logger.warning(f"predicted: {pred}")
    
    return True


test(iris.data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys
predicted: ['setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'setosa', 'setosa', 'versicolor', 'versicolor', 'virginica', 'versicolor', 'versicolor', 'set

True